In [6]:
import requests
import csv
from tqdm import tqdm
import re

In [2]:
with open("../data/sentences_en.csv", "r") as f:
    dict_reader = csv.DictReader(f, delimiter=",")
    sentences = list(dict_reader)

In [3]:
def get_wikidata(wikipage):
    response = requests.get(
        url = "https://www.wikidata.org/w/api.php?action=wbgetentities&format=json&sites=enwiki&titles="+wikipage
        )
    try:
        response.raise_for_status()
    except requests.exceptions.HTTPError as e:
        return None
    else:
        response = response.json()
        wbitem = list(response["entities"].keys())[0]
        if wbitem == "-1":
            response = requests.get(
                url = "https://www.wikidata.org/w/api.php?action=wbgetentities&format=json&sites=enwiki&titles="+wikipage+"&normalize=1"
            )
            response = response.json()
            wbitem = list(response["entities"].keys())[0]
        return wbitem

In [10]:
output = []
url = "http://10.10.4.10:22002"
pbar = tqdm(total=len(sentences))
for sentence in sentences:
    text = sentence["sentence"]
    text = re.sub("\.", "", text)
    data = '[{"text":"'+text+'"}]'
    r = requests.post(url, data=data)
    try:
        entities = r.json()[0]["disambiguated_entities"]
        # print(entities)
        for entity in entities:
            item = {"id":sentence["id"],
            "start_pos":entity["char_start"],
            "end_pos":entity["char_end"],
            "surface":entity["mention"],
            "alias":entity["entity"],
            "wb_id":get_wikidata(entity["entity"])}
            output.append(item)
        pbar.update(1)
    except KeyError:
        print(data)
        print(r.json())
        pbar.update(1)
        continue

pbar.close()

keys = output[0].keys()

a_file = open("../results/extend_en/output_nel.csv", "w")
dict_writer = csv.DictWriter(a_file, keys)
dict_writer.writeheader()
dict_writer.writerows(output)
a_file.close()


 70%|█████████████████████████████▉             | 23/33 [01:37<00:29,  2.98s/it]

[{"text":"In S Maria Maggiore in Florence, beside the Chapel of the Panciatichi, there is a very beautiful Pietà with little figures"}]
{'detail': 'There was an error parsing the body'}


 76%|████████████████████████████████▌          | 25/33 [01:39<00:17,  2.15s/it]

[{"text":"At this time Sandro was commissioned to paint a little panel with figures three-quarters of a braccio in length, which was placed between two doors in the principal façade of S Maria Novella, on the left as one enters the church by the door in the centre It contains the Adoration of the Magi, and wonderful feeling is seen in the first old man, who, kissing the foot of Our Lord, and melting with tenderness, shows very clearly that he has achieved the end of his long journey"}]
{'detail': 'There was an error parsing the body'}


100%|███████████████████████████████████████████| 33/33 [02:04<00:00,  3.76s/it]
